# Kaggle-based Text Classification


In [1]:
import nltk
from urllib.request import urlopen
import urllib.parse
import string
from nltk.util import ngrams
import re
from nltk.corpus import brown
from nltk.stem.wordnet import WordNetLemmatizer
import random

import pandas as pd
# import cPickle as pickle
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
from pandas.tools.plotting import scatter_matrix
from ast import literal_eval 
from collections import defaultdict
import datetime

# analysis
# import statsmodels.api as sm
# from scipy.stats import pearsonr
# from sklearn.cross_validation import train_test_split, cross_val_score
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
# from sklearn.metrics import roc_curve, auc, classification_report
# from sklearn.preprocessing import normalize, scale, StandardScaler, Normalizer
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.grid_search import GridSearchCV

# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
raw_data = pd.read_csv('yelp_data_official_training.csv', sep='|', low_memory=False)
beauty_spa = ""
home_services = ""
health_medical = ""
local_services = ""
veterinarians = ""
pets = ""

reviews_tmp = list(raw_data['Review Text'])
categories = list(raw_data['Category'])
for i in range(len(categories)):
    if categories[i] == 1:
        beauty_spa += " " + str(reviews_tmp[i])
    elif categories[i] == 2:
        home_services += " " + str(reviews_tmp[i])
    elif categories[i] == 3:
        health_medical += " " + str(reviews_tmp[i])
    elif categories[i] == 4:
        local_services += " " + str(reviews_tmp[i])
    elif categories[i] == 5:
        veterinarians += " " + str(reviews_tmp[i])
    elif categories[i] == 6:
        pets += str(reviews_tmp[i])

reviews = [beauty_spa, home_services, health_medical, local_services, veterinarians, pets]


In [3]:
def tokenize_text(corpus):
    '''Split text into sentences and tokenize
    '''
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus.replace("\ufeff", "")) 
    return [nltk.word_tokenize(word) for word in raw_sents]


def build_stop_words():
    '''Build stop words from SMART (Salton,1971).  Available at ftp://ftp.cs.cornell.edu/pub/smart/english.stop
    '''
    stop_words = []
    file = "SmartStoplist.txt"
    try:
        with open(file, 'r') as fp:
            tmp = fp.readlines()
            stop_words = [ word.replace('\n', '') for word in tmp ]
    except:
        print("Can't open specified file: {0}".format(file))
        
    return stop_words

def extract_unigram(sents):
    '''Extract unigram candidates, and prune the candidates.

    '''
    unigram_raw_candidates = []
    unigram_candidates = []
    lm = WordNetLemmatizer()
    for sent in sents:
        unigram_raw_candidates += list(ngrams(sent,1))    
    
    # Unigram pruning: remove punctions, stop words, words that are capitalized in the first character, words less than 2 characters.
    stopwords = build_stop_words()
  
    unigram_pattern = r"\d+|\'+|\`+|^[A-Z]\w*|\w+\'\w+|\.+"
    for element in unigram_raw_candidates:
        if re.match(unigram_pattern, element[0]) or element[0] in string.punctuation or element[0] in stopwords or len(element[0])<2 :
            continue
        else:
            unigram_candidates.append(lm.lemmatize(element[0], 'v'))
    return unigram_candidates


In [4]:
category_data = []

i = 0
for element in range(6):
    for word in set(extract_unigram(tokenize_text(reviews[i]))):
        category_data.append(({'unigram': word}, i+1))
    i += 1
random.shuffle(category_data)

print(len(category_data))


81622


In [5]:
cl = nltk.NaiveBayesClassifier.train(category_data[:40000])
print ("%.3f" % nltk.classify.accuracy(cl, category_data[40001:]))

0.115


In [6]:
test_data = pd.read_csv('yelp_data_official_test_nocategories.csv', sep='|', low_memory=False)

In [7]:
def write_file(content):
    with open('first-submit.csv', 'a') as fp:
        for (n, v) in content:
            line = str(n) + ',' + str(v)
            fp.write(line)
            fp.write('\n')
        fp.close()
        
id = 0
content = []
for test_reviews in list(test_data['Review Text']):
    extract_unigrams = set(extract_unigram(tokenize_text(test_reviews)))
    results = []
    for element in extract_unigrams:
        features = {
            'unigram': element
        }
        results.append(cl.classify(features))
    for (category, counts) in nltk.FreqDist(results).most_common(1):
        content.append((id, category))
    id += 1
    
write_file(content)